# LearningPySpark_Ch03
## Chapter 3: DataFrames
This notebook contains sample code from Chapter 4 of <a href="https://render.githubusercontent.com/view/ipynb?commit=edcaf5ed42558fb08759f70a01201e00aa2d49f7&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f5061636b745075626c697368696e672f4c6561726e696e672d5079537061726b2f656463616635656434323535386662303837353966373061303132303165303061613264343966372f4368617074657230322f4c6561726e696e675079537061726b5f4368617074657230332e6970796e62&nwo=PacktPublishing%2FLearning-PySpark&path=Chapter02%2FLearningPySpark_Chapter03.ipynb&repository_id=83264169">Learning PySpark</a> focusing on PySpark and DataFrames.

 - A DataFrame is an immutable distributed collection of data that is organized into named columns analogous to a table in a relation database
 - Python Pandas DataFrame or R DataFrame, a Spark DataFrame is a similar concept in that it allows users to easily work with structured data 
  - some differenceses as well so please temper your expectations.
 - specifically, the Catalyst Optimizer - to significantly improve the performance of Spark queries. 
  - In earlier APIs of Spark(that's RDDs), executing queries in python could be significantly slower due to communication overhead between the Java JVM and Py4J.
 

### Python to RDD communications

 - PySpark program is executed using RDDs, there's potentially large overhead to execute the job
  - Whenever PySpark execute some code with RDDs, PySpark Driver, the $Spark Context$ uses $Py4j$ 'JVM' using the JavaSparkContext. 
 - Any RDD trasnformations are mapped to PythonRDD objects in java
 - these tasks are pushed out to the Spark Worker(s), PythonRDD objects launch Python subprocesses using pipes to send both code and data to be processed within Python:

![img1](img/1.jpg)

### Catalyst Optimizer refresh
 - one of the primary reasons the Spark SQL engine is so fast is because of the $Catalyst$ $Optimizer$.
![img1](img/1.PNG)

## Creating DataFrames
### Generate your own DataFrame
- Instead of accessing the file system, let's create a DataFrame by generating the data. 
- In this case, we'll first create the stringRDD RDD and then convert it into a DataFrame when we're reading stringJSONRDD using spark.read.json.

In [1]:
# Generate our own JSON data 
#   This way we don't have to access the file system yet.
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Simone",
    "age": 23,
    "eyeColor": "blue"
  }""")
)
stringJSONRDD

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:475

In [2]:
stringJSONRDD.collect()

[' \n  { "id": "123",\n    "name": "Katie",\n    "age": 19,\n    "eyeColor": "brown"\n  }',
 '{\n    "id": "234",\n    "name": "Michael",\n    "age": 22,\n    "eyeColor": "green"\n  }',
 '{\n    "id": "345",\n    "name": "Simone",\n    "age": 23,\n    "eyeColor": "blue"\n  }']

#### Create DataFrame

In [5]:
# Create DataFrame
swimmersJSON = spark.read.json(stringJSONRDD)
swimmersJSON.collect()

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

#### Create temporary table

In [6]:
# Create temporary table
swimmersJSON.createOrReplaceTempView("swimmersJSON")

### Simple DataFrame Queries
#### DataFrame API Query

In [7]:
# DataFrame API
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



#### SQL Query
 - $.collect()$ methond, which returns all the records as a list of Row objects
 - $collect()$ or $show()$ method for both DataFrames and SQL queries
 - $.collect()$ is for a small DataFrame, shince it will return all of the rows in the DataFrame and move them back from the executors to the driver
  - $take(<n>)$ or $show(<n>)$, allow you to limit the number of rows returned by specifying.

In [8]:
# SQL Query
spark.sql("select * from swimmersJSON").collect()

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

In [9]:
# if you are using Databricks, i can use %sql command
#%sql 
#-- Query Data
#select * from swimmersJSON

### Interoperating with RDDs
There are two different methods for converting existing RDDs to DataFrame(or Datasets)
 - inferring the schema using reflection, or programmatically specifying the schema

#### Inferring the Schema Using Reflection
Note that Apache Spark is inferring the schema using reflection; <br>
i.e. it automaticlaly determines the schema of the data based on reviewing the JSON data.

In [10]:
# Print the schema
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



##### but What if we don't know or that's not the same type ?
 - the id is actually a long instead of a string?

 - Notice that Spark was able to determine infer the schema (when reviewing the schema using .printSchema).
 - But what if we want to programmatically specify the schema?

#### Programmatically Specifying the Schema
In this case, let's specify the schema for a CSV text file.

In [11]:
from pyspark.sql.types import *

# Generate our own CSV data 
#   This way we don't have to access the file system yet.
stringCSVRDD = sc.parallelize([(123, 'Katie', 19, 'brown'), (234, 'Michael', 22, 'green'), (345, 'Simone', 23, 'blue')])
stringCSVRDD.collect()

[(123, 'Katie', 19, 'brown'),
 (234, 'Michael', 22, 'green'),
 (345, 'Simone', 23, 'blue')]

#### User define Structure and Variable's Type
 - StructField class
  - name = The name of this field
  - dataType = The data type of this field
  - nullable = Indicates whether values of this field can be null 
 - Almost Same things like RDBMS

In [12]:
# The schema is encoded in a string, using StructType we define the schema using various pyspark.sql.types
schemaString = "id name age eyeColor"
schema = StructType([
    StructField("id", LongType(), True),    
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

In [13]:
# Apply the schema to the RDD and Create DataFrame
swimmers = spark.createDataFrame(stringCSVRDD, schema)

In [14]:
# Creates a temporary view using the DataFrame
swimmers.createOrReplaceTempView("swimmers")

In [15]:
# Print the schema
#   Notice that we have redefined id as Long (instead of String)
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



In [16]:
#%sql 
#-- Query the data
#select * from swimmers

In [17]:
spark.sql("select * from swimmers").show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



As you can see from above, we can programmatically apply the schema instead of allowing the Spark engine to infer the schema via reflection.

Additional Resources include:
 - <a href="https://spark.apache.org/docs/2.0.0/api/python/pyspark.sql.html">PySpark API Reference</a>
 - <a href="https://spark.apache.org/docs/latest/sql-programming-guide.html#programmatically-specifying-the-schema">Spark SQL, DataFrames, and Datasets Guide </a>: This is in reference to Programmatically Specifying the Schema using a CSV file.

|| SparkSession

Notice that we're no longer using sqlContext.read... but instead spark.read.... This is because as part of Spark 2.0, HiveContext,  SQLContext, StreamingContext, SparkContext have been merged together into the Spark Session spark.
 - Entry point for reading data
 - Working with metadata
 - Configuration
 - Cluster resource management

For more information, please refer to How to use <a href="http://bit.ly/2br0Fr1">SparkSession</a> in Apache Spark 2.0 (http://bit.ly/2br0Fr1).

## Querying with the DataFrame API
 - can start off by using collect(), show(), or take() to view the data within DataFrame

#### Number of Rows 

In [20]:
swimmers.count()

3

In [21]:
type(swimmers)

pyspark.sql.dataframe.DataFrame

#### Running filter statements

In [20]:
# Query id and age for swimmers with age =22 via Dataframe API
swimmers.select("id","age").filter("age=22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [23]:
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age==22).show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [24]:
#Query name and eye color for swimmers with eye color starting with the letter 'b'
swimmers.select("name","eyeColor").filter("eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



### Querying with SQL (SQL engine)
 - Recall that this DataFrame is accessible because we executed the $.createOrReplaceTempView$ method for $swimmers$

With DataFrames, you can start writing your queries using Spark SQL - a SQL dialect that is compatible with the Hive Query Language (or HiveQL).

In [16]:
# SQL문 실행 / 데이터 출력 
spark.sql("select * from swimmers").show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



 #### Number of Rows

In [17]:
spark.sql("select count(*) from swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



#### Running filter statements using the where Clauses

In [21]:
# Query id and age for swimmers with age = 22 in SQL 
spark.sql("select id, age from swimmers where age =22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [23]:
#Query name and eye color for swimmers with eye color starting with the letter 'b'
spark.sql("select name, eyeColor from swimmers where eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



### DataFrame scenario – on-time flight performance (DataSets)
 - Airline On-Time Performance and Causes of Flight Delays: On-Time Data (http://bit.ly/2ccJPPM)

In [25]:
# set file Path
flightPerfFilePath = "./data/flight/departuredelays.csv"
airportsFilePath = "./data/flight/airport-codes-na.txt"

#### Obtain Airports dataset

In [27]:
airports = spark.read.csv(airportsFilePath,header=True,inferSchema=True, sep='\t')
airports.createOrReplaceTempView("airports") # make tables

In [32]:
airports.take(5)

[Row(City='Abbotsford', State='BC', Country='Canada', IATA='YXX'),
 Row(City='Aberdeen', State='SD', Country='USA', IATA='ABR'),
 Row(City='Abilene', State='TX', Country='USA', IATA='ABI'),
 Row(City='Akron', State='OH', Country='USA', IATA='CAK'),
 Row(City='Alamosa', State='CO', Country='USA', IATA='ALS')]

#### Obtain Departure Delays dataset

In [30]:
flightPerf = spark.read.csv(flightPerfFilePath, header=True)
flightPerf.createOrReplaceTempView("FlightPerformance")

In [31]:
flightPerf.take(5)

[Row(date='01011245', delay='6', distance='602', origin='ABE', destination='ATL'),
 Row(date='01020600', delay='-8', distance='369', origin='ABE', destination='DTW'),
 Row(date='01021245', delay='-2', distance='602', origin='ABE', destination='ATL'),
 Row(date='01020605', delay='-4', distance='602', origin='ABE', destination='ATL'),
 Row(date='01031245', delay='-4', distance='602', origin='ABE', destination='ATL')]

#### Cache the Departure Delays dataset
 - finally, we cachethe flight dataset so subsequent queries will be faster.

In [33]:
flightPerf.cache()

DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]

### Joining flight performance and airports
 - One of the most common tasks with Dataframes / SQL is to join two different datasets with identified columns 
  - 

#### Query Sum of Flight Delays by City and Origin Code
 - the total delays by city and origin code for the state of Washington

In [34]:
spark.sql("""select a.City, f.origin, sum(f.delay) as Delays 
from FlightPerformance f join airports a 
on a.IATA = f.origin where a.State = 'WA'
group by a.City,f.origin 
order by Delays desc""").show()

+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+



 - Using notebooks (such as Databricks, iPython, Jupyter, and Apache Zeppelin), you can more easily execute and visualize your queries
![img2](img/2.PNG)

### Visualizing our flight-performance data

In [35]:
'''
 %sql
-- Query Sum of Flight Delays by State (for the US)
select a.State, sum(f.delay) as Delays
from FlightPerformance f
join airports a
on a.IATA = f.origin
where a.Country = 'USA'
group by a.State
'''

"\n %sql\n-- Query Sum of Flight Delays by State (for the US)\nselect a.State, sum(f.delay) as Delays\nfrom FlightPerformance f\njoin airports a\non a.IATA = f.origin\nwhere a.Country = 'USA'\ngroup by a.State\n"

![img3](img/3.PNG)

### Spark Dataset API
![img4](img/4.PNG)

## Summary
 - One of the main reasons Python is initially slower within Spark is due to the communication layer between Python sub-processes and the JVM.
 - Spark DataFrames has many performance enhancements through the Catalyst Optimizer and Project Tungsten which we have reviewed in this chapter.
 - we also reviewed how to work with Spark DataFrames and worked on an on-time flight performance scenario using DataFrames